<img src="https://github.com/ShiJbey/neighborly/assets/11076525/3013afe0-2bcd-4908-bddf-31adcd8cbd7f" />

# Sample Simulation: Demon Slayer

**Demon Slayer: Kimetsu no Yaiba** is a manga/anime series about the aventures of the main protagonist, Kamado Tanjiro, and his sister Kamado Nezuko. The story is set during the Taisho era in Japan (1912-1926), and focuses on a centuries-old conflict between the Demon Slayer Corps and demons. Demons prey on humans during the night causing fear amoung humans. The Demon Slayer Corp fights demons to eradicate the threat. To fight the supernatural powers/strength of demons, demon slayers use elemental *breathing styles*. The strongest Demon Slayers are known as the Hashira, and gain this rank through either killing a member of the Twelve Kizuki (the twelve strongest demons under Muzan) or killing fifty (50) demons after ranking up multiple times.

This sample simulates a Demon Slayer-style town where humans live generally-mundate lives (growing up, forming relationships, working jobs, having families, etc.). However, at a deignated point in the simulation, we introduce a single demon into the population. This demon is free to turn human into demons or devour them to gain strength. Ordinary humans will have the option to become demon slayers and fight against demons. We simulate demons and demon slayers growing in rank. Demons have the option to challenge someone of higher ranking for their position.

We simulate Era-specific business types and occupations. These were initially generated using ChatGPT, then hand modified.


## Key Features
- Regular people can turn into Demons
- Demons are immortal and will age, but cannot die
- Regular people can become DemonSlayers
- As Demons and DemonSlayers defeat enemies,
  they grow in rank
- Demons can challenge higher-ranking demons for power
- The DemonSlayerCorp tracks the current highest
  ranking slayers
- The DemonKingdom tracks the top-12 ranked demons


## Copyright Disclaimer

This project is associated with the Demon Slayer series. All characters and story-specific terms are the property of their current copyright holder.

## References

- [https://en.wikipedia.org/wiki/Demon_Slayer:_Kimetsu_no_Yaiba](https://en.wikipedia.org/wiki/Demon_Slayer:_Kimetsu_no_Yaiba)
- [https://fictionhorizon.com/demon-slayer-ranks-in-order-every-rank-explained-and-sorted/](https://fictionhorizon.com/demon-slayer-ranks-in-order-every-rank-explained-and-sorted/)
- [https://kimetsu-no-yaiba.fandom.com/wiki/Breathing_Styles](https://kimetsu-no-yaiba.fandom.com/wiki/Breathing_Styles)
- [https://kimetsu-no-yaiba.fandom.com/wiki/Demon_Slayer_Corps#Positions](https://kimetsu-no-yaiba.fandom.com/wiki/Demon_Slayer_Corps#Positions)
- [https://kimetsu-no-yaiba.fandom.com/wiki/Twelve_Kizuki](https://kimetsu-no-yaiba.fandom.com/wiki/Twelve_Kizuki)
- [https://demonslayerrp.fandom.com/wiki/Slayer_Rankings](https://demonslayerrp.fandom.com/wiki/Slayer_Rankings)


# Import Dependencies

In [ ]:
from __future__ import annotations

import math
import random
import time
from enum import IntEnum
from typing import Any, Dict, List, Optional, Tuple

from ordered_set import OrderedSet

from neighborly import (
    Component,
    GameObject,
    IComponentFactory,
    ISystem,
    Neighborly,
    NeighborlyConfig,
    SimDateTime,
    World,
)
from neighborly.command import SpawnCharacter
from neighborly.components.character import (
    CanAge,
    CanGetPregnant,
    GameCharacter,
    LifeStage,
    LifeStageType,
)
from neighborly.components.shared import FrequentedLocations
from neighborly.core.ai.brain import ConsiderationList
from neighborly.core.ecs import Active, ISerializable
from neighborly.core.life_event import EventRole, EventRoleList, RandomLifeEvent
from neighborly.core.settlement import Settlement
from neighborly.decorators import (
    component,
    component_factory,
    on_event,
    random_life_event,
    resource,
    system,
)
from neighborly.events import DeathEvent
from neighborly.exporter import export_to_json
from neighborly.plugins.defaults.actions import Die
from neighborly.systems import EarlyUpdateSystemGroup
from neighborly.utils.common import add_character_to_settlement

# Create Neighborly Simulation Instance

We configure the simulation tick size to be one month ("1mo") and we configure relationships to track frienship (platonic affinity) and romance (romanic affinity).

In [ ]:
sim = Neighborly(
    NeighborlyConfig.parse_obj(
        {
            "time_increment": "1mo",
            "relationship_schema": {
                "components": {
                    "Friendship": {
                        "min_value": -100,
                        "max_value": 100,
                    },
                    "Romance": {
                        "min_value": -100,
                        "max_value": 100,
                    },
                    "InteractionScore": {
                        "min_value": -5,
                        "max_value": 5,
                    },
                }
            }
        }
    )
)

# Defining component types

Here we define simulation-specific component types that track information that we care about for this simulation.

In [ ]:
class DemonSlayerRank(IntEnum):
    """Various ranks within the DemonSlayerCorp by increasing seniority."""

    Mizunoto = 0
    Mizunoe = 1
    Kanoto = 2
    Kanoe = 3
    Tsuchinoto = 4
    Tsuchinoe = 5
    Hinoto = 6
    Hinoe = 7
    Kinoto = 8
    Kinoe = 9
    Hashira = 10


class BreathingStyle(IntEnum):
    """Various breathing styles for demon slayers."""

    Flower = 0
    Love = 1
    Flame = 2
    Sun = 3
    Sound = 4
    Thunder = 5
    Wind = 6
    Water = 7
    Insect = 8
    Serpent = 9
    Moon = 10
    Stone = 11
    Mist = 12
    Beast = 13


@component(sim.world)
class PowerLevel(Component, ISerializable):
    """The strength of a character when in battle."""

    __slots__ = "level"

    level: int

    def __init__(self, level: int = 0) -> None:
        super().__init__()
        self.level = level

    def to_dict(self) -> Dict[str, Any]:
        return {"level": self.level}


@component(sim.world)
class ConfirmedKills(Component, ISerializable):
    """The number of enemies a character has defeated."""

    __slots__ = "count"

    count: int

    def __init__(self, count: int = 0) -> None:
        super().__init__()
        self.count = count

    def to_dict(self) -> Dict[str, Any]:
        return {"count": self.count}


@component(sim.world)
class DemonSlayer(Component):
    """A DemonSlayer is a character who fights demons and grows in rank."""

    __slots__ = "rank", "breathing_style"

    rank: DemonSlayerRank
    """The slayer's current rank."""

    breathing_style: BreathingStyle
    """The style of breathing this slayer uses."""

    def __init__(self, breathing_style: BreathingStyle) -> None:
        super().__init__()
        self.rank = DemonSlayerRank.Mizunoto
        self.breathing_style = breathing_style

    def to_dict(self) -> Dict[str, Any]:
        return {"rank": self.rank.name, "breathing-style": self.breathing_style.name}


@component_factory(sim.world, DemonSlayer)
class DemonSlayerFactory(IComponentFactory):
    """Creates instances of DemonSlayer Components."""

    def create(self, world: World, **kwargs: Any) -> DemonSlayer:
        rng = world.resource_manager.get_resource(random.Random)
        breathing_style = rng.choice(list(BreathingStyle))
        return DemonSlayer(breathing_style=breathing_style)


@component(sim.world)
class DemonSlayerCorps:
    """Tracks information about current and former Hashira."""

    __slots__ = "_hashira", "_former_hashira", "_hashira_styles", "_hashira_to_style"
    
    _hashira: OrderedSet[GameObject]
    """Currently serving Hashira"""
    
    _hashira_styles: OrderedSet[Breathing Style]
    """Breathing styles of current Hashira"""
    
    _hashira_to_style: Dict[GameObject, BreathingStyle] = {}
    """Map of Hashira IDs to Breathing styles"""
        
    _former_hashira: OrderedSet[GameObject] = OrderedSet([])
    """Hashira that are no longer serving"""
    
    def __init__(self) -> None:
        self._hashira = OrderedSet([])
        self._hashira_styles = OrderedSet([])
        self._hashira_to_style = {}
        self._former_hashira = OrderedSet([])

    @property
    def hashira(self) -> OrderedSet[int]:
        """Get all current Hashira"""
        return self._hashira

    @property
    def former_hashira(self) -> OrderedSet[int]:
        """Get all former Hashira"""
        return self._former_hashira

    def add_hashira(self, gid: int, breathing_style: BreathingStyle) -> None:
        """
        Add a new Hashira

        Parameters
        ----------
        gid: int
            GameObject ID of the new Hashira
        breathing_style: BreathingStyle
            The BreathingStyle of the new Hashira
        """
        self._hashira.add(gid)
        self._hashira_styles.add(breathing_style)

    def retire_hashira(self, gid: int) -> None:
        """
        Remove a hashira from the active list

        Parameters
        ----------
        gid: int
            GameObject of the Hashira to remove
        """
        self._hashira.remove(gid)
        self._former_hashira.add(gid)
        self._hashira_styles.remove(self._hashira_to_style[gid])
        del self._hashira_to_style[gid]

    def has_vacancy(self, breathing_style: BreathingStyle) -> bool:
        """
        Return true if there is a vacancy for a Hashira
        with the given BreathingStyle

        Parameters
        ----------
        breathing_style: BreathingStyle
            The BreathingStyle to see if there is a vacancy for
        """
        return breathing_style in self._hashira_styles

    def to_dict(self) -> Dict[str, Any]:
        return {
            "hashira": list(self.hashira),
            "former_hashira": list(self.former_hashira),
        }


class DemonRank(IntEnum):
    """The various ranks held by demons"""

    LowerDemon = 0
    Demon = 1
    BloodDemon = 2
    HigherDemon = 3
    SuperiorDemon = 4
    LowerMoon = 5
    UpperMoon = 6


@component(sim.world)
class Demon(Component):
    """
    Demons eat people and battle each other and demon slayers

    Attributes
    ----------
    rank: DemonRank
        This demon's rank among other demons
    turned_by: Optional[int]
        GameObject ID of the demon that gave this demon
        their power
    """

    __slots__ = "rank", "turned_by"

    def __init__(
        self,
        rank: DemonRank = DemonRank.LowerDemon,
        turned_by: Optional[int] = None,
    ) -> None:
        super().__init__()
        self.rank: DemonRank = rank
        self.turned_by: Optional[int] = turned_by

    def to_dict(self) -> Dict[str, Any]:
        return {
            "rank": str(self.rank.name),
            "turned_by": self.turned_by if self.turned_by else -1,
        }


@resource(sim.world)
class DemonKingdom:
    """
    Manages information about the upper-ranked demons

    Attributes
    ----------
    _upper_moons: OrderedSet[int]
        The demons ranked as UpperMoon
    _former_upper_moons: OrderedSet[int]
        The demons that were formerly ranked as UpperMoon
    _lower_moons: OrderedSet[int]
        The demons ranked as LowerMoon
    _former_lower_moons: OrderedSet[int]
        The demons that were formerly ranked as LowerMoon
    """

    MAX_UPPER_MOONS = 6
    MAX_LOWER_MOONS = 6

    __slots__ = (
        "_lower_moons",
        "_upper_moons",
        "_former_upper_moons",
        "_former_lower_moons",
    )

    def __init__(self) -> None:
        self._lower_moons: OrderedSet[int] = OrderedSet([])
        self._upper_moons: OrderedSet[int] = OrderedSet([])
        self._former_lower_moons: OrderedSet[int] = OrderedSet([])
        self._former_upper_moons: OrderedSet[int] = OrderedSet([])

    @property
    def upper_moons(self) -> OrderedSet[int]:
        """Get current Upper Moons"""
        return self._upper_moons

    @property
    def lower_moons(self) -> OrderedSet[int]:
        """Get current Lower Moons"""
        return self._upper_moons

    @property
    def former_upper_moons(self) -> OrderedSet[int]:
        """Get former Upper Moons"""
        return self._former_upper_moons

    @property
    def former_lower_moons(self) -> OrderedSet[int]:
        """Get former Lower Moons"""
        return self._former_upper_moons

    def add_lower_moon(self, gid: int) -> None:
        """Add a new LowerMoon demon"""
        self._lower_moons.add(gid)

    def retire_lower_moon(self, gid: int) -> None:
        """Remove LowerMoon demon"""
        self._lower_moons.remove(gid)

    def add_upper_moon(self, gid: int) -> None:
        """Add a new UpperMoon demon"""
        self._upper_moons.add(gid)

    def retire_upper_moon(self, gid: int) -> None:
        """Remove UpperMoon demon"""
        self._upper_moons.remove(gid)

    def has_lower_moon_vacancy(self) -> bool:
        """Return true if there are sport available as LowerMoons"""
        return len(self._lower_moons) < DemonKingdom.MAX_LOWER_MOONS

    def has_upper_moon_vacancy(self) -> bool:
        """Return true if there are sport available as LowerMoons"""
        return len(self._upper_moons) < DemonKingdom.MAX_UPPER_MOONS

    def to_dict(self) -> Dict[str, Any]:
        return {
            "lower_moons": list(self.lower_moons),
            "former_lower_moons": list(self.lower_moons),
            "upper_moons": list(self.upper_moons),
            "former_upper_moons": list(self.former_upper_moons),
        }

# Defining Constants

Here we define values for the ELO system used to rank demons and the power level requirements for demon slayers and demons to reach a specific rank.

In [ ]:
# ELO parameters used to update power levels
ELO_SCALE: int = 255
ELO_K: int = 16

# Maximum power level for a demon or demon slayer
POWER_LEVEL_MAX: int = 255

# Minimum power levels for each demon slayer rank
MIZUNOTO_PL: int = 0
MIZUNOE_PL: int = 15
KANOTO_PL: int = 30
KANOE_PL: int = 50
TSUCHINOTO_PL: int = 65
TSUCHINOE_PL: int = 80
HINOTO_PL: int = 100
HINOE_PL: int = 120
KINOTO_PL: int = 140
KINOE_PL: int = 160
HASHIRA_PL: int = 220

# Minimum power levels for each demon rank
LOWER_DEMON_PL: int = 0
DEMON_PL: int = 40
BLOOD_DEMON_PL: int = 80
HIGHER_DEMON_PL: int = 120
SUPERIOR_DEMON_PL: int = 160
LOWER_MOON_PL: int = 200
UPPER_MOON_PL: int = 220

# Utility Functions for ELO and Ranking

In [ ]:
def probability_of_winning(rating_a: int, rating_b: int) -> float:
    """Calculate the probability of GameObject A defeating GameObject B.

    Parameters
    ----------
    rating_a
        The ELO rating of GameObject A.
    rating_b: int
        The ELO rating of GameObject B.
    
    Returns
    -------
    float
        A probability on the interval [0, 1.0]
    """
    return 1.0 / (1 + math.pow(10, (rating_a - rating_b) / ELO_SCALE))


def update_power_level(
    winner_rating: int,
    loser_rating: int,
    winner_expectation: float,
    loser_expectation: float,
    k: int = 16,
) -> Tuple[int, int]:
    """Perform ELO calculation for new ELO ratings following a match.

    Parameters
    ----------
    winner_rating
        The current ELO rating of the winner.
    loser_rating
        The current ELO rating of the loser.
    winner_expectation
        The expected probablity of the winner winning.
    loser_expectation
        The expected probability of the loser losing.
        This should be (1 - winner_expectation).
    k
        A calculation constant used in ELO calculations.

    Returns
    -------
    Tuple[int, int]
        The new ELO ratings for the winner and loser, respectively.
    """
    new_winner_rating: int = round(winner_rating + k * (1 - winner_expectation))
    new_winner_rating = min(POWER_LEVEL_MAX, max(0, new_winner_rating))
    new_loser_rating: int = round(loser_rating + k * (0 - loser_expectation))
    new_loser_rating = min(POWER_LEVEL_MAX, max(0, new_loser_rating))

    return new_winner_rating, new_loser_rating


def power_level_to_slayer_rank(power_level: int) -> DemonSlayerRank:
    """Convert a power level to the corresponding demon slayer rank.
    
    Parameters
    ----------
    power_level
        The current power level of a demon slayer.
        
    Returns
    -------
    DemonSlayerRank
        The rank thats associated with this power level.
    """
    if power_level >= HASHIRA_PL:
        return DemonSlayerRank.Hashira
    elif power_level >= KINOE_PL:
        return DemonSlayerRank.Kinoe
    elif power_level >= KINOTO_PL:
        return DemonSlayerRank.Kinoto
    elif power_level >= HINOE_PL:
        return DemonSlayerRank.Hinoe
    elif power_level >= HINOTO_PL:
        return DemonSlayerRank.Hinoto
    elif power_level >= TSUCHINOE_PL:
        return DemonSlayerRank.Tsuchinoe
    elif power_level >= TSUCHINOTO_PL:
        return DemonSlayerRank.Tsuchinoto
    elif power_level >= KANOE_PL:
        return DemonSlayerRank.Kanoe
    elif power_level >= KANOTO_PL:
        return DemonSlayerRank.Kanoto
    elif power_level >= MIZUNOE_PL:
        return DemonSlayerRank.Mizunoe
    else:
        return DemonSlayerRank.Mizunoto


def power_level_to_demon_rank(power_level: int) -> DemonRank:
    """Convert a power level to the corresponding Demon rank.
    
    Parameters
    ----------
    power_level
        The current power level of a demon.

    Returns
    -------
    DemonRank
        The rank thats associated with this power level.
    """
    if power_level >= UPPER_MOON_PL:
        return DemonRank.UpperMoon
    elif power_level >= LOWER_MOON_PL:
        return DemonRank.LowerMoon
    elif power_level >= SUPERIOR_DEMON_PL:
        return DemonRank.SuperiorDemon
    elif power_level >= HIGHER_DEMON_PL:
        return DemonRank.HigherDemon
    elif power_level >= BLOOD_DEMON_PL:
        return DemonRank.BloodDemon
    elif power_level >= DEMON_PL:
        return DemonRank.Demon
    else:
        return DemonRank.LowerDemon

In [ ]:
def run_sim_for(sim: Neighborly, years: int) -> None:
    """Run the simulation for a given number of years and display a progrss bar.
    
    Parameters
    ----------
    sim
        A Neighborly simulation instance.
    years
        The number of years to simulate.
    """
    
    # tqdm requires needs to know the number of iterations beforehand.
    # So, we need to calculate how many timesteps are in the given number
    # of years (rounding up to the nearest whole number)
    total_timesteps = math.ceil(
        float(years * HOURS_PER_YEAR) / sim.config.time_increment.total_hours
    )
    
    for _ in tqdm(range(total_timesteps)):
        sim.step()

# Define Random Events

Here we define random events that can happen to characters. Random events are also another way of defining character behavior. We use these events to build emergent stories.

In [ ]:
@random_life_event(sim.world)
class BecomeDemonSlayer(RandomLifeEvent):
    considerations: Dict[str, ConsiderationList] = {
        "Character": ConsiderationList([lambda gameobject: 0.8])
    }

    def __init__(self, date: SimDateTime, character: GameObject) -> None:
        super().__init__(date, [EventRole("Character", character)])

    def get_probability(self) -> float:
        return self.considerations["Character"].calculate_score(self["Character"])

    def execute(self) -> None:
        character = self["Character"]
        world = character.world
        character.add_component(
            world.get_component_info(DemonSlayer.__name__).factory.create(world)
        )
        character.add_component(
            PowerLevel(
                world.resource_manager.get_resource(random.Random).randint(0, HINOTO_PL)
            )
        )
        character.add_component(ConfirmedKills())

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        # Only create demon slayers if demons are an actual problem
        bindings = bindings if bindings is not None else EventRoleList()

        demons_exist = len(world.get_component(Demon)) > 5
        if demons_exist is False:
            return None

        character = cls._bind_character(world, bindings.get_first_or_none("Character"))

        if character:
            return cls(world.resource_manager.get_resource(SimDateTime), character)

        return None

    @staticmethod
    def _bind_character(world: World, candidate: Optional[GameObject] = None):
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((GameCharacter, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            if character.has_component(DemonSlayer):
                continue
            if character.has_component(Demon):
                continue
            if character.get_component(LifeStage).life_stage >= LifeStageType.Child:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None


@random_life_event(sim.world)
class DemonSlayerPromotion(RandomLifeEvent):
    def __init__(self, date: SimDateTime, character: GameObject) -> None:
        super().__init__(date, [EventRole("Character", character)])

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        character = self["Character"]
        slayer = character.get_component(DemonSlayer)
        power_level_rank = power_level_to_slayer_rank(
            character.get_component(PowerLevel).level
        )
        slayer.rank = power_level_rank

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()
        character = cls._bind_demon_slayer(
            world, bindings.get_first_or_none("Character")
        )

        if character is None:
            return None

        cls(world.resource_manager.get_resource(SimDateTime), character)

    @staticmethod
    def _bind_demon_slayer(world: World, candidate: Optional[GameObject] = None):
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((DemonSlayer, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            demon_slayer = character.get_component(DemonSlayer)
            power_level = character.get_component(PowerLevel)
            power_level_rank = power_level_to_slayer_rank(power_level.level)

            if power_level_rank < demon_slayer.rank:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None


@random_life_event(sim.world)
class DemonChallengeForPower(RandomLifeEvent):
    def __init__(
        self, date: SimDateTime, challenger: GameObject, opponent: GameObject
    ) -> None:
        super().__init__(
            date, [EventRole("Challenger", challenger), EventRole("Opponent", opponent)]
        )

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        """Execute the battle"""
        challenger = self["Challenger"]
        opponent = self["Opponent"]
        world = challenger.world

        battle_event = Battle(
            world.resource_manager.get_resource(SimDateTime), challenger, opponent
        )

        world.event_manager.dispatch_event(battle_event)

        battle_event.execute(world)

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()

        challenger = cls._bind_challenger(
            world, bindings.get_first_or_none("Challenger")
        )

        if challenger is None:
            return None

        opponent = cls._bind_opponent(
            world, challenger, bindings.get_first_or_none("Opponent")
        )

        if opponent is None:
            return None

        return cls(
            world.resource_manager.get_resource(SimDateTime), challenger, opponent
        )

    @staticmethod
    def _bind_challenger(world: World, candidate: Optional[GameObject] = None):
        """Get a challenger demon that has someone above them"""

        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            demon = character.get_component(Demon)
            if demon.rank < DemonRank.UpperMoon:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None

    @staticmethod
    def _bind_opponent(
        world: World, challenger: GameObject, candidate: Optional[GameObject]
    ):
        """Find an opponent for the challenger"""
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            if character == challenger:
                continue

            if (
                character.get_component(Demon).rank
                > challenger.get_component(Demon).rank
            ):
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None


@random_life_event(sim.world)
class DevourHuman(RandomLifeEvent):
    def __init__(
        self, date: SimDateTime, demon: GameObject, victim: GameObject
    ) -> None:
        super().__init__(date, [EventRole("Demon", demon), EventRole("Victim", victim)])

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        demon = self["Demon"]
        victim = self["Victim"]
        date = self.world.resource_manager.get_resource(SimDateTime)

        if victim.has_component(DemonSlayer):
            battle_event = Battle(date, demon, victim)
            battle_event.dispatch()

        else:
            demon.get_component(PowerLevel).level += 1
            demon.get_component(Demon).rank = power_level_to_demon_rank(
                demon.get_component(PowerLevel).level
            )
            Die(victim).evaluate()

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()

        demon = cls._bind_demon(world, bindings.get_first_or_none("Demon"))

        if demon is None:
            return None

        victim = cls._bind_victim(world, demon, bindings.get_first_or_none("Victim"))

        if victim is None:
            return None

        return cls(world.resource_manager.get_resource(SimDateTime), demon, victim)

    @staticmethod
    def _bind_demon(world: World, candidate: Optional[GameObject] = None):
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        if candidates:
            return world.resource_manager.get_resource(random.Random).choice(candidates)

        return None

    @staticmethod
    def _bind_victim(
        world: World, demon: GameObject, candidate: Optional[GameObject] = None
    ):
        """Get all people at the same location who are not demons"""
        demon_frequented_locations = OrderedSet(
            demon.get_component(FrequentedLocations)
        )

        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((GameCharacter, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            if character == demon:
                continue

            if character.has_component(Demon):
                # skip
                continue

            character_frequented = OrderedSet(
                character.get_component(FrequentedLocations)
            )

            shared_locations = demon_frequented_locations.intersection(
                character_frequented
            )

            if len(shared_locations) > 0:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None


@random_life_event(sim.world)
class Battle(RandomLifeEvent):
    """Have a demon fight a demon slayer"""

    def __init__(
        self, date: SimDateTime, challenger: GameObject, opponent: GameObject
    ) -> None:
        super().__init__(
            date, [EventRole("Challenger", challenger), EventRole("Opponent", opponent)]
        )

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        """Choose a winner based on their expected success"""
        challenger = self["Challenger"]
        opponent = self["Opponent"]
        world = challenger.world
        rng = world.resource_manager.get_resource(random.Random)
        challenger_pl = challenger.get_component(PowerLevel)
        opponent_pl = opponent.get_component(PowerLevel)

        challenger_success_chance = probability_of_winning(
            challenger_pl.level, opponent_pl.level
        )

        opponent_success_chance = probability_of_winning(
            opponent_pl.level, challenger_pl.level
        )

        if rng.random() < challenger_success_chance:
            # Challenger wins
            new_challenger_pl, _ = update_power_level(
                challenger_pl.level,
                opponent_pl.level,
                challenger_success_chance,
                opponent_success_chance,
            )

            challenger_pl.level = new_challenger_pl

            Die(opponent).evaluate()

            challenger.get_component(ConfirmedKills).count += 1
        else:
            # Opponent wins
            _, new_opponent_pl = update_power_level(
                opponent_pl.level,
                challenger_pl.level,
                opponent_success_chance,
                challenger_success_chance,
            )

            opponent_pl.level = new_opponent_pl

            Die(challenger).evaluate()

            opponent.get_component(ConfirmedKills).count += 1

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()

        challenger = cls._bind_challenger(
            world, bindings.get_first_or_none("Challenger")
        )
        opponent = cls._bind_opponent(world, bindings.get_first_or_none("Opponent"))

        if challenger is None:
            return None

        if opponent is None:
            return None

        return cls(
            world.resource_manager.get_resource(SimDateTime), challenger, opponent
        )

    @staticmethod
    def _bind_challenger(world: World, candidate: Optional[GameObject] = None):
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        if candidates:
            return world.resource_manager.get_resource(random.Random).choice(candidates)

        return None

    @staticmethod
    def _bind_opponent(world: World, candidate: Optional[GameObject] = None):
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((DemonSlayer, Active))
            ]

        if candidates:
            return world.resource_manager.get_resource(random.Random).choice(candidates)

        return None


@random_life_event(sim.world)
class TurnSomeoneIntoDemon(RandomLifeEvent):
    def __init__(
        self, date: SimDateTime, demon: GameObject, new_demon: GameObject
    ) -> None:
        super().__init__(
            date, [EventRole("Demon", demon), EventRole("NewDemon", new_demon)]
        )

    def get_probability(self) -> float:
        return 0.8

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()

        demon = cls._bind_demon(world, candidate=bindings.get_first_or_none("Demon"))

        if demon is None:
            return None

        new_demon = cls._bind_new_demon(
            world, demon, candidate=bindings.get_first_or_none("NewDemon")
        )

        if new_demon is None:
            return None

        return cls(world.resource_manager.get_resource(SimDateTime), demon, new_demon)

    @staticmethod
    def _bind_demon(
        world: World, candidate: Optional[GameObject]
    ) -> Optional[GameObject]:
        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        if candidates:
            return world.resource_manager.get_resource(random.Random).choice(candidates)

        return None

    @staticmethod
    def _bind_new_demon(
        world: World, demon: GameObject, candidate: Optional[GameObject]
    ) -> Optional[GameObject]:
        demon_frequented_locations = OrderedSet(
            demon.get_component(FrequentedLocations)
        )

        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((GameCharacter, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            if character == demon:
                continue

            if character.has_component(Demon):
                # skip
                continue

            character_frequented = OrderedSet(
                character.get_component(FrequentedLocations)
            )

            shared_locations = demon_frequented_locations.intersection(
                character_frequented
            )

            if len(shared_locations) > 0:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None

    def execute(self) -> None:
        demon = self["Demon"]
        new_demon = self["NewDemon"]

        new_demon.add_component(Demon(turned_by=demon.uid))
        new_demon.remove_component(CanAge)
        if new_demon.has_component(CanGetPregnant):
            new_demon.remove_component(CanGetPregnant)
        new_demon.add_component(PowerLevel(demon.get_component(PowerLevel).level // 2))
        new_demon.add_component(ConfirmedKills())


@random_life_event(sim.world)
class PromotionToLowerMoon(RandomLifeEvent):
    def __init__(self, date: SimDateTime, character: GameObject) -> None:
        super().__init__(date, [EventRole("Character", character)])

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        character = self["Character"]
        demon = character.get_component(Demon)
        demon.rank = DemonRank.LowerMoon

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        bindings = bindings if bindings is not None else EventRoleList()

        demon = cls._bind_demon(world, bindings.get_first_or_none("Character"))
        if demon:
            return cls(world.resource_manager.get_resource(SimDateTime), demon)
        return None

    @staticmethod
    def _bind_demon(world: World, candidate: Optional[GameObject] = None):
        demon_kingdom = world.resource_manager.get_resource(DemonKingdom)

        if not demon_kingdom.has_lower_moon_vacancy():
            return None

        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            demon = character.get_component(Demon)
            power_level = character.get_component(PowerLevel)
            if demon.rank < DemonRank.LowerMoon and power_level.level >= LOWER_MOON_PL:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None


@random_life_event(sim.world)
class PromotionToUpperMoon(RandomLifeEvent):
    def __init__(self, date: SimDateTime, character: GameObject) -> None:
        super().__init__(date, [EventRole("Character", character)])

    def get_probability(self) -> float:
        return 0.8

    def execute(self) -> None:
        character = self["Character"]
        demon = character.get_component(Demon)
        demon.rank = DemonRank.UpperMoon

    @classmethod
    def instantiate(
        cls,
        world: World,
        bindings: Optional[EventRoleList] = None,
    ) -> Optional[RandomLifeEvent]:
        if bindings:
            demon = cls._bind_demon(world, bindings.get_first_or_none("Character"))
        else:
            demon = cls._bind_demon(world)

        if demon:
            return cls(world.resource_manager.get_resource(SimDateTime), demon)
        return None

    @staticmethod
    def _bind_demon(world: World, candidate: Optional[GameObject] = None):
        demon_kingdom = world.resource_manager.get_resource(DemonKingdom)

        if not demon_kingdom.has_upper_moon_vacancy():
            return None

        candidates: List[GameObject]
        if candidate:
            candidates = [candidate]
        else:
            candidates = [
                world.gameobject_manager.get_gameobject(g)
                for g, _ in world.get_components((Demon, Active))
            ]

        matches: List[GameObject] = []

        for character in candidates:
            demon = character.get_component(Demon)
            power_level = character.get_component(PowerLevel)
            if demon.rank < DemonRank.UpperMoon and power_level.level >= UPPER_MOON_PL:
                matches.append(character)

        if matches:
            return world.resource_manager.get_resource(random.Random).choice(matches)

        return None




# Define event handlers

Sometimes we want to know when certain events happen to characters. We may want to update its components or the components of other GameObjects to reflect this event. Here we listen for `DeathEvent`s and handle them differently based on if they are a Hashira or a Demon.

In [ ]:
@on_event(sim.world, DeathEvent)
def handle_hashira_death(world: World, event: DeathEvent) -> None:
    if demon_slayer := event.character.try_component(DemonSlayer):
        if demon_slayer.rank == DemonSlayerRank.Hashira:
            event.character.world.resource_manager.get_resource(
                DemonSlayerCorps
            ).retire_hashira(event.character.uid)


@on_event(sim.world, DeathEvent)
def handle_demon_death(world: World, event: DeathEvent) -> None:
    if demon := event.character.try_component(Demon):
        if demon.rank == DemonRank.LowerMoon:
            event.character.world.resource_manager.get_resource(
                DemonKingdom
            ).retire_lower_moon(event.character.uid)
        elif demon.rank == DemonRank.UpperMoon:
            event.character.world.resource_manager.get_resource(
                DemonKingdom
            ).retire_upper_moon(event.character.uid)

# Spawn the first demon

We create a simple system that chooses an active character in the simulation and turns them into a demon. From here, they are free to make demon choices and engage in demon behavior.

In [ ]:
@system(sim.world, system_group=EarlyUpdateSystemGroup)
class SpawnFirstDemon(ISystem):
    def on_update(self, world: World) -> None:
        date = self.world.resource_manager.get_resource(SimDateTime)

        if date.year < 10:
            return

        for guid, _ in self.world.get_component(Settlement):
            settlement = self.world.gameobject_manager.get_gameobject(guid)

            new_demon = (
                SpawnCharacter("character::default::female")
                .execute(self.world)
                .get_result()
            )

            new_demon.add_component(Demon())
            new_demon.remove_component(CanAge)
            new_demon.add_component(ConfirmedKills())
            new_demon.add_component(
                PowerLevel(
                    self.world.resource_manager.get_resource(random.Random).randint(
                        0, HIGHER_DEMON_PL
                    )
                )
            )
            if new_demon.has_component(CanGetPregnant):
                new_demon.remove_component(CanGetPregnant)

            add_character_to_settlement(new_demon, settlement)

        self.world.remove_system(type(self))

# Track Population Size

A simple system to create is one that tracks the population size each year. We define it as an ECS system that runs during the early-update phase of a simulation step and counts the number of active characters in the simulation

In [ ]:
data_collector.create_new_table("population", ("timestamp", "population"))


class TrackPopulation(ISystem):
    sys_group = "early-update"
    priority = 9999
    
    last_recorded_year = 0
    
    def process(self, *args, **kwargs):
        dc = self.world.get_resource(DataCollector)
        current_date = self.world.get_resource(SimDateTime)
        
        if current_date.year > self.last_recorded_year:
            population = len(self.world.get_components((Active, GameCharacter)))
            dc.add_table_row("population", {"timestamp": current_date.year, "population": population})
            self.last_recorded_year = current_date.year

# Run the Simulation!

In [ ]:
run_sim_for(sim, years=500)

# Visualize the Population Over Time

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
plt.plot(
    data_collector.tables["population"]["timestamp"],
    data_collector.tables["population"]["population"]
)
plt.xlabel("Year")
plt.ylabel("Population")
plt.title(f"Population of Simulation (seed: {sim.config.seed})")